In [25]:
import os

# Append '/usr/bin/' to the PATH
os.environ['PATH'] += os.pathsep + '/usr/bin/'

In [26]:
import subprocess
import sys
import json
from vosk import Model, KaldiRecognizer

# Path to your Vosk model
model_path = "/workspaces/vosk-api/python/renzo/model1"  # Update with your actual model path

# Initialize the Vosk model
model = Model(model_path)

# Create a KaldiRecognizer with the model and sampling rate
recognizer = KaldiRecognizer(model, 16000)

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:11:12:13:14:15
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /workspaces/vosk-api/python/renzo/model1/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /workspaces/vosk-api/python/renzo/model1/graph/HCLr.fst /workspaces/vosk-api/python/renzo/model1/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /workspaces/vosk-api/python/renzo/model1/graph/phones/word_boundary.int


In [27]:
# Command to capture audio from Twitch
twitch_channel = "twitch_channel_name"  # Replace with the Twitch channel name

streamlink_command = [
    "streamlink",
    f"twitch.tv/{twitch_channel}",
    "audio_only",
    "--stdout",  # Output to stdout
]

ffmpeg_path = '/usr/bin/ffmpeg'  # Replace with the actual path from 'which ffmpeg'
ffmpeg_command = [
    "ffmpeg",
    "-loglevel", "quiet",  # Suppress ffmpeg output if desired
    "-i", "pipe:0",        # Input from stdin
    "-ac", "1",            # Mono audio
    "-ar", "16000",        # Sampling rate 16000 Hz
    "-f", "s16le",         # Output format
    "pipe:1",              # Output to stdout
]

# Start the streamlink process
streamlink_process = subprocess.Popen(
    streamlink_command,
    stdout=subprocess.PIPE,
    stderr=subprocess.DEVNULL,
)

# Start the ffmpeg process
ffmpeg_process = subprocess.Popen(
    ffmpeg_command,
    stdin=streamlink_process.stdout,  # Get input from streamlink process
    stdout=subprocess.PIPE,
    stderr=subprocess.DEVNULL,
)

# Ensure that streamlink_process.stdout doesn't get closed when ffmpeg_process exits
streamlink_process.stdout.close()

# Read audio data and transcribe
try:
    while True:
        data = ffmpeg_process.stdout.read(4000)
        if len(data) == 0:
            break

        if recognizer.AcceptWaveform(data):
            result = recognizer.Result()
            text = json.loads(result)["text"]
            if text:
                print(f"Transcript: {text}")
        else:
            partial_result = recognizer.PartialResult()
            partial_text = json.loads(partial_result)["partial"]
            # You can print partial results if desired
            # print(f"Partial: {partial_text}")
except KeyboardInterrupt:
    pass
finally:
    ffmpeg_process.kill()
    streamlink_process.kill()

In [28]:
twitch_channel = "renzoscriber" 

Use this to create a `test_output.ts` file:

`streamlink twitch.tv/renzoscriber audio_only --stdout > test_output.ts`

In [29]:
import subprocess
import sys
import json
from vosk import Model, KaldiRecognizer
import os
import ffmpeg

# Ensure ffmpeg is in PATH
os.environ['PATH'] += os.pathsep + '/usr/bin/'

# Verify PATH (optional)
print(os.environ['PATH'])

# Path to your Vosk model
model_path = "/workspaces/vosk-api/python/renzo/model1"  # Update with your actual model path

# Initialize the Vosk model
model = Model(model_path)

# Create a KaldiRecognizer with the model and sampling rate
recognizer = KaldiRecognizer(model, 16000)

# FFmpeg command to read from test_output.ts instead of live stream
ffmpeg_command = [
    'ffmpeg',
    "-i", "/workspaces/vosk-api/test_output0.ts",     # Use the .ts file as input
    "-ac", "1",                 # Mono audio
    "-ar", "16000",             # Sampling rate 16000 Hz
    "-f", "s16le",              # Output format
    "pipe:1",                   # Output to stdout
]

# Start the ffmpeg process to convert the .ts file to raw audio
ffmpeg_process = subprocess.Popen(
    ffmpeg_command,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,   # Capture stderr for debugging
)

# Read audio data and transcribe
try:
    while True:
        data = ffmpeg_process.stdout.read(4000)
        if len(data) == 0:
            break

        if recognizer.AcceptWaveform(data):
            result = recognizer.Result()
            text = json.loads(result)["text"]
            if text:
                print(f"Transcript: {text}")
        else:
            partial_result = recognizer.PartialResult()
            partial_text = json.loads(partial_result)["partial"]
            # You can print partial results if desired
            # print(f"Partial: {partial_text}")
except KeyboardInterrupt:
    pass
finally:
    # Read and print any errors from ffmpeg before closing and killing the process
    ffmpeg_errors = ffmpeg_process.stderr.read().decode('utf-8')
    if ffmpeg_errors:
        print(f"FFmpeg Error: {ffmpeg_errors}")

    # Close and kill the process
    ffmpeg_process.stderr.close()
    ffmpeg_process.stdout.close()
    ffmpeg_process.kill()

/workspaces/vosk-api/.venv/bin:/vscode/bin/linux-x64/fabdb6a30b49f79a7aba0f2ad9df9b399473380f/bin/remote-cli:/home/codespace/.local/bin:/home/codespace/.dotnet:/home/codespace/nvm/current/bin:/home/codespace/.php/current/bin:/home/codespace/.python/current/bin:/home/codespace/java/current/bin:/home/codespace/.ruby/current/bin:/home/codespace/.local/bin:/usr/local/python/current/bin:/usr/local/py-utils/bin:/usr/local/oryx:/usr/local/go/bin:/go/bin:/usr/local/sdkman/bin:/usr/local/sdkman/candidates/java/current/bin:/usr/local/sdkman/candidates/gradle/current/bin:/usr/local/sdkman/candidates/maven/current/bin:/usr/local/sdkman/candidates/ant/current/bin:/usr/local/rvm/gems/default/bin:/usr/local/rvm/gems/default@global/bin:/usr/local/rvm/rubies/default/bin:/usr/local/share/rbenv/bin:/usr/local/php/current/bin:/opt/conda/bin:/usr/local/nvs:/usr/local/share/nvm/current/bin:/usr/local/hugo/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/share/dotnet:/home/codespace/.dot

LOG (VoskAPI:ReadDataFiles():model.cc:213) Decoding params beam=13 max-active=7000 lattice-beam=6
LOG (VoskAPI:ReadDataFiles():model.cc:216) Silence phones 1:2:3:4:5:11:12:13:14:15
LOG (VoskAPI:RemoveOrphanNodes():nnet-nnet.cc:948) Removed 0 orphan nodes.
LOG (VoskAPI:RemoveOrphanComponents():nnet-nnet.cc:847) Removing 0 orphan components.
LOG (VoskAPI:ReadDataFiles():model.cc:248) Loading i-vector extractor from /workspaces/vosk-api/python/renzo/model1/ivector/final.ie
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:183) Computing derived variables for iVector extractor
LOG (VoskAPI:ComputeDerivedVars():ivector-extractor.cc:204) Done.
LOG (VoskAPI:ReadDataFiles():model.cc:282) Loading HCL and G from /workspaces/vosk-api/python/renzo/model1/graph/HCLr.fst /workspaces/vosk-api/python/renzo/model1/graph/Gr.fst
LOG (VoskAPI:ReadDataFiles():model.cc:308) Loading winfo /workspaces/vosk-api/python/renzo/model1/graph/phones/word_boundary.int


Transcript: the
Transcript: the
Transcript: my name
Transcript: okay
Transcript: so
Transcript: we don't want to look at that
Transcript: i was i i was kind of successful tonight
Transcript: where is where my results
Transcript: haha my transcript from my test stream
Transcript: that a she
Transcript: it says that thing okay us and canada stream discussion to discontinue it he
Transcript: this is what is unsupported will result in incoherent output data oh no is
Transcript: how appropriate okay that's okay that is totally fine are do remember saying that so let me see here okay so what did i want to see oh
Transcript: i know okay so it looks like i have good what's it called streaming no not streaming wifi whatever sorry on a little poo brain and that's how i knew it was definitely me but i'm excited i think i got it to work knock on wood i i'm going to end
Transcript: this dream see what happens when i do that
Transcript: and then i
Transcript: and then i ended the stream
Transcript: 